In [1]:
import math
import torch
from torch import nn
from torch import Tensor
from torch.nn  import functional as F
import gpytorch
from matplotlib import pyplot as plt
import sys
from decimal import Decimal
sys.path.append("..")
import vvkernels as vvk
import sep_vvkernels as svvk
import vvk_rbfkernel as vvk_rbf
import vvmeans as vvm
import vvlikelihood as vvll
from vfield import VField
from LBFGS import FullBatchLBFGS
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
vf = VField()
f_target = vf.tgt_vec
sample_size = 100
D = vf.D
N = vf.N

def g_theta(sample_size, D):
    loc = np.random.random_sample((sample_size,D))
    return Tensor(loc)
train_x = g_theta(sample_size, D)
train_y = torch.zeros([sample_size, N])




for i in range(sample_size):
    train_y[i] = Tensor(vf(train_x[i])) #+ torch.randn(Tensor(vf(train_x[i])).size()) * 0.02
    

In [3]:
def vfield_(x):
    x = x.reshape(x.shape[0],D)
    out = torch.zeros(x.shape[0], N)
    for i in range(x.shape[0]):
        out[i] = Tensor(vf(x[i])) #+ torch.randn(Tensor(vf(x[i])).size()) * 0.02
    return out

In [4]:
def stopping_criteria(tol_vector, f_target, lower_bound, upper_bound):
    lower_tol_vector = f_target - tol_vector
    upper_tol_vector = f_target + tol_vector
    SUCCESS = True
    FAILURE = False
    for i in range(f_target.shape[0]):
            if (lower_bound[i] < lower_tol_vector[i]) or  (upper_bound[i] > upper_tol_vector[i]):
                SUCCESS = False  
            if ((lower_bound[i] > upper_tol_vector[i]) or  (upper_bound[i] < lower_tol_vector[i])):
                FAILURE = True
    return SUCCESS, FAILURE

In [5]:

x_train = train_x #loc #torch.linspace(0, 1, 10)
y_train = train_y #v  #torch.stack([torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,torch.cos(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,], -1)

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood,num_base_kernels):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        a = torch.ones(2,2)
        chol_q = torch.tril(a)
        self.mean_module = vvm.TensorProductSubMean(gpytorch.means.ConstantMean(), num_tasks = 2)
        base_kernels = []
        for i in range(num_base_kernels):
            base_kernels.append(vvk_rbf.vvkRBFKernel())


        self.covar_module = svvk.SepTensorProductKernel(base_kernels,num_tasks = 2)

#\         self.covar_module = gpytorch.kernels.MultitaskKernel(
#             gpytorch.kernels.RBFKernel(), num_tasks=2, rank=1
#         )
       # self.covar_module = vvk.TensorProductKernel(vvk_rbf.vvkRBFKernel(), a[0,0], a[1,0], a[1,1], num_tasks = 2, rank =1,  task_covar_prior=None)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x,x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)
    
    



In [6]:
# # ###hyperparameters optimization###
# def hyper_opti(g_theta1, agg_data, training_iter):
#     noises = torch.ones(agg_data.shape[0]) * 0.0001
#     likelihood = vvll.FixedNoiseMultitaskGaussianLikelihood(2, noises)

#     model = MultitaskGPModel(g_theta1, agg_data, likelihood,3)
#     model.train()
#     likelihood.train()

#     optimizer = torch.optim.Adam(model.parameters(),  lr=0.1)  # Includes GaussianLikelihood parameters
#     mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
#     for i in range(training_iter):
#         optimizer.zero_grad()
#         output = model(g_theta1)
#         #output_ll = likelihood(output)

#         #loss = -likelihood.get_mll(agg_data,output_ll)
#         loss = -mll(output, agg_data)
#         loss.backward(retain_graph=True)

#         print('Iter %d/%d - Loss hyperparam: %.3f' % (i + 1, training_iter, loss.item()))
#         optimizer.step()

#     return model, likelihood


In [7]:
def hyper_opti(g_theta1, agg_data, training_iter,num_base_kernels):
    noises = torch.ones(agg_data.shape[0]) * 0.001
    likelihood = vvll.TensorProductLikelihood(num_tasks = 2)

    model = MultitaskGPModel(g_theta1, agg_data, likelihood,num_base_kernels)
    model.train()
    likelihood.train()
    optimizer = FullBatchLBFGS(model.parameters(), lr=.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    def closure():
        
        optimizer.zero_grad()
        output = model(g_theta1)
        #output_ll = likelihood(output)

        #loss = -likelihood.get_mll(agg_data,output_ll)
        loss = -mll(output, agg_data)
        print('Loss gp: %.3f' % ( loss))
        
        return loss
    
    loss = closure()
    loss.backward()

    for i in range(training_iter):
        options = {'closure': closure, 'current_loss': loss, 'max_ls': 10}
        loss, _, _, _, _, _, _, fail = optimizer.step(options)

        if fail:
            print('Convergence reached!')
            break
        #print(i)
    return model, likelihood

In [8]:
class design_opti(nn.Module):
    def __init__(self, sample, x):
        super(design_opti, self).__init__()
        #loc = np.random.random_sample((loc_size,2))
        self.g_theta2 = nn.Parameter(Tensor(sample))
        self.x_design = nn.Parameter(Tensor(x))
    def forward(self):
       
        g_theta2_new = self.g_theta2 #filter_sample(self.g_theta2, 0.009)
        
        return (g_theta2_new), self.x_design

In [9]:
def conduct_design_opti(x0,loc_sample, f_target, g_theta1, agg_data, model, likelihood, training_design_iter, training_param_iter, lr_new):
    design = design_opti(loc_sample,x0)
    loc_sample0 = loc_sample
    
    g_theta2, x_d = design.forward()
    def closure():
        optimizer.zero_grad()
        
        loss2,lower_bound, upper_bound = likelihood.get_ell(agg_data,f_target,x_d, g_theta1, g_theta2, model, likelihood)
        loss2 = -1. * loss2
        loss2.backward(retain_graph=True)
#         print(x_d)
#         print(lower_bound)
#         print(upper_bound)
       
        return loss2
        
        
        
    optimizer = torch.optim.LBFGS(design.parameters(), lr=lr_new, history_size=100, max_iter=100, line_search_fn="strong_wolfe")
    optimizer.step(closure)

    g_theta2, x_d = design.forward()
    loss2,lower_bound, upper_bound = likelihood.get_ell(agg_data,f_target,x_d, g_theta1, g_theta2, model, likelihood)
    loss2 = -1. * loss2
    print('Loss design: %.3f' % ( loss2))
    #print(x_d)
    return x_d, g_theta2, lower_bound, upper_bound
    
    

In [10]:
iter_hp = 100
iter_design = 40 
iter_param = 50
num_base_kernels = 3


f_target = Tensor(vf.tgt_vec) 
f_target = f_target.reshape(f_target.shape[0],1) 
tol_vector = 0.001 * torch.ones(f_target.shape)


loc_size = 10
loc_sample = np.random.random_sample((loc_size,2))
g_theta2_vec = (Tensor(loc_sample).clone()).flatten()

g_theta1 = x_train
agg_data = y_train.flatten()


x0 = Tensor(np.array([0.5,0.7])) 
x0 = x0.reshape(1,2)
x00 = x0 
vec_x = Tensor(np.array([0.5,0.7])) 
vec_x = vec_x.reshape(1,2)

lr_new = 1.

SUCCESS = False 
FAILURE = False 
iter = 0 
tol = 0.009 
while(SUCCESS == False and iter < 50):
    print('START HYPERPARAMETERS optimization')
    model, likelihood = hyper_opti(g_theta1,agg_data,iter_hp,num_base_kernels)

    print('END HYPERPARAMETERS optimization')
    print(iter)
    
    x0_new,g_theta2,lower_bound, upper_bound = conduct_design_opti(x0, loc_sample, f_target, g_theta1, agg_data, model, likelihood, iter_design,iter_param, lr_new)
    g_theta2_vec = torch.cat([g_theta2_vec, g_theta2.flatten()], 0)
    print(torch.norm(upper_bound - tol_vector))
#     if (torch.norm(upper_bound - tol_vector) <= 0.1 ):
#         print('bkhjghf')
#         lr_new = lr_new * 0.1
    print(lower_bound)
    print(upper_bound)
    print(f_target-tol_vector)
    print(f_target+tol_vector)
    loc_sample = np.random.random_sample((loc_size,2))
    x0 = x0_new #Tensor(np.random.random_sample((1,2))) #x0_new
    vec_x = torch.cat([vec_x, x0_new])
    g_theta2_detach =  g_theta2.detach()
    new_data = vfield_(g_theta2_detach)
    agg_data = torch.cat([agg_data, new_data.flatten()], 0)
    g_theta1= torch.cat([g_theta1, g_theta2], 0)

    SUCCESS, FAILURE = stopping_criteria(tol_vector, f_target, lower_bound, upper_bound)
    iter = iter + 1
    print(x0_new)

START HYPERPARAMETERS optimization
Loss gp: 0.870
Loss gp: 0.858
Loss gp: 0.847
Loss gp: 0.824
Loss gp: 0.777
Loss gp: 0.680
Loss gp: 0.473
Loss gp: 0.025
Loss gp: -0.919
Loss gp: -2.555
Loss gp: -2.911
Loss gp: -2.946
Loss gp: -2.956
Loss gp: -2.963
Loss gp: -2.974
Loss gp: -2.984
Loss gp: -2.996
Loss gp: -3.005
Loss gp: -3.013
Loss gp: -3.019
Loss gp: -3.023
Loss gp: -3.028
Loss gp: -3.032
Loss gp: -3.036
Loss gp: -3.039
Loss gp: -3.041
START HYPERPARAMETERS optimization
Loss gp: 0.870
Loss gp: 0.858
Loss gp: 0.847
Loss gp: 0.824
Loss gp: 0.777
Loss gp: 0.680
Loss gp: 0.473
Loss gp: 0.025
Loss gp: -0.919
Loss gp: -2.555
Loss gp: -2.911
Loss gp: -2.946
Loss gp: -2.956
Loss gp: -2.963
Loss gp: -2.974
Loss gp: -2.984
Loss gp: -2.996
Loss gp: -3.005
Loss gp: -3.013
Loss gp: -3.019
Loss gp: -3.023
Loss gp: -3.028
Loss gp: -3.032
Loss gp: -3.036
Loss gp: -3.039
Loss gp: -3.041
Loss gp: -3.044
Loss gp: -3.045
Loss gp: -3.049
Loss gp: -3.052
Loss gp: -3.056
Loss gp: -3.059
Loss gp: -3.064
Lo

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gpytorch/utils/cholesky.py:83: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new:.1e} to the diagonal", NumericalWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gpytorch/utils/cholesky.py:83: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new:.1e} to the diagonal", NumericalWarning)


Loss design: -11.529
tensor(1.1229, grad_fn=<CopyBackwards>)
tensor([[0.4955],
        [0.9953]], grad_fn=<CopySlices>)
tensor([[0.5046],
        [1.0047]], grad_fn=<CopySlices>)
tensor([[0.4990],
        [0.9990]])
tensor([[0.5010],
        [1.0010]])
Parameter containing:
tensor([[0.2010, 0.0996]], requires_grad=True)
START HYPERPARAMETERS optimization
Loss gp: 0.853
Loss gp: 0.842
Loss gp: 0.830
Loss gp: 0.807
Loss gp: 0.759
Loss gp: 0.660
Loss gp: 0.450
Loss gp: -0.007
Loss gp: -0.967
Loss gp: -2.636
Loss gp: -3.030
Loss gp: -3.063
Loss gp: -3.073
Loss gp: -3.082
Loss gp: -3.087
Loss gp: -3.091
Loss gp: -3.095
Loss gp: -3.099
Loss gp: -3.103
Loss gp: -3.104
Loss gp: -3.108
Loss gp: -3.109
Loss gp: -3.112
Loss design: -11.529
tensor(1.1229, grad_fn=<CopyBackwards>)
tensor([[0.4955],
        [0.9953]], grad_fn=<CopySlices>)
tensor([[0.5046],
        [1.0047]], grad_fn=<CopySlices>)
tensor([[0.4990],
        [0.9990]])
tensor([[0.5010],
        [1.0010]])
Parameter containing:
tensor(

Loss gp: -3.259
Loss gp: -3.260
Loss gp: -3.260
Loss gp: -3.261
Loss gp: -3.261
Loss gp: -3.261
Loss gp: -3.261
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Loss gp: -3.262
Convergence reached!
END HYPERPARAMETERS optimization
3
Loss design: -11.844
tensor(1.1224, grad_fn=<CopyBackwards>)
tensor([[0.4959],
        [0.9959]], grad_fn=<CopySlices>)
tensor([[0.5042],
        [1.0043]], grad_fn=<CopySlices>)
tensor([[0.4990],
        [0.9990]])
tensor([[0.5010],
        [1.0010]])
Parameter containing:
tensor([[0.2005, 0.0997]], requires_grad=True)
START HYPERPARAMETERS optimization
Loss gp: 0.842
Loss gp: 0.830
Loss gp: 0.819
Loss gp: 0.795
Loss gp: 0.746
Loss gp: 0.646
Loss gp: 0.431
Loss gp: -0.035
Loss gp: -1.017
Loss gp: -2.703
Loss gp: -3.091
Loss gp: -3.103
Loss gp: -3.113
Loss gp: -3.130
Lo

Loss gp: -3.319
Loss gp: -3.319
Loss gp: -3.319
Loss gp: -3.319
Loss gp: -3.319
Loss gp: -3.319
Loss gp: -3.319
Loss gp: -3.319
Loss gp: -3.319
Loss gp: -3.319
Loss gp: -3.319
Convergence reached!
END HYPERPARAMETERS optimization
6
Loss design: -12.031
tensor(1.1218, grad_fn=<CopyBackwards>)
tensor([[0.4964],
        [0.9965]], grad_fn=<CopySlices>)
tensor([[0.5036],
        [1.0039]], grad_fn=<CopySlices>)
tensor([[0.4990],
        [0.9990]])
tensor([[0.5010],
        [1.0010]])
Parameter containing:
tensor([[0.2016, 0.1007]], requires_grad=True)
START HYPERPARAMETERS optimization
Loss gp: 0.825
Loss gp: 0.813
Loss gp: 0.801
Loss gp: 0.777
Loss gp: 0.728
Loss gp: 0.625
Loss gp: 0.406
Loss gp: -0.070
Loss gp: -1.075
Loss gp: -2.795
Loss gp: -3.150
Loss gp: -3.158
Loss gp: -3.163
Loss gp: -3.172
Loss gp: -3.181
Loss design: -12.031
tensor(1.1218, grad_fn=<CopyBackwards>)
tensor([[0.4964],
        [0.9965]], grad_fn=<CopySlices>)
tensor([[0.5036],
        [1.0039]], grad_fn=<CopySlices>)

...

Loss design: -12.669
tensor(1.1203, grad_fn=<CopyBackwards>)
tensor([[0.4975],
        [0.9976]], grad_fn=<CopySlices>)
tensor([[0.5026],
        [1.0027]], grad_fn=<CopySlices>)
tensor([[0.4990],
        [0.9990]])
tensor([[0.5010],
        [1.0010]])
Parameter containing:
tensor([[0.2009, 0.1002]], requires_grad=True)
START HYPERPARAMETERS optimization
Loss gp: 0.776
Loss gp: 0.763
Loss gp: 0.750
Loss gp: 0.725
Loss gp: 0.750
Loss gp: 0.725
Loss gp: 0.673
Loss gp: 0.564
Loss gp: 0.673
Loss gp: 0.564
Loss gp: 0.331
Loss gp: -0.176
Loss gp: 0.331
Loss gp: -0.176
Loss gp: -1.249
Loss gp: -3.065
Loss gp: -1.249
Loss gp: -3.065
Loss gp: -3.350
Loss gp: -3.359
Loss gp: -3.350
Loss gp: -3.359
Loss gp: -3.368
Loss gp: -3.380
Loss gp: -3.368
Loss gp: -3.380
Loss gp: -3.390
Loss gp: -3.399
Loss gp: -3.390
Loss gp: -3.399
Loss gp: -3.408
Loss gp: -3.416
Loss gp: -3.408
Loss gp: -3.416
Loss gp: -3.424
Loss gp: -3.432
Loss gp: -3.424
Loss gp: -3.432
Loss gp: -3.440
Loss gp: -3.448
Loss gp: -3.440

Loss gp: -3.527
Loss gp: -3.527
Loss gp: -3.527
Convergence reached!
END HYPERPARAMETERS optimization
40
Loss gp: -3.527
Loss gp: -3.527
Loss gp: -3.527
Convergence reached!
END HYPERPARAMETERS optimization
40
Loss design: -12.639
tensor(1.1198, grad_fn=<CopyBackwards>)
tensor([[0.4973],
        [0.9972]], grad_fn=<CopySlices>)
tensor([[0.5023],
        [1.0023]], grad_fn=<CopySlices>)
tensor([[0.4990],
        [0.9990]])
tensor([[0.5010],
        [1.0010]])
Parameter containing:
tensor([[0.2009, 0.0997]], requires_grad=True)
START HYPERPARAMETERS optimization
Loss gp: 0.772
Loss gp: 0.760
Loss design: -12.639
tensor(1.1198, grad_fn=<CopyBackwards>)
tensor([[0.4973],
        [0.9972]], grad_fn=<CopySlices>)
tensor([[0.5023],
        [1.0023]], grad_fn=<CopySlices>)
tensor([[0.4990],
        [0.9990]])
tensor([[0.5010],
        [1.0010]])
Parameter containing:
tensor([[0.2009, 0.0997]], requires_grad=True)
START HYPERPARAMETERS optimization
Loss gp: 0.772
Loss gp: 0.760
Loss gp: 0.747
L

Loss gp: -3.502
Loss gp: -3.507
Loss gp: -3.512
Loss gp: -3.516
Loss gp: -3.512
Loss gp: -3.516
Loss gp: -3.518
Loss gp: -3.521
Loss gp: -3.518
Loss gp: -3.521
Loss gp: -3.523
Loss gp: -3.524
Loss gp: -3.523
Loss gp: -3.524
Loss gp: -3.526
Loss gp: -3.527
Loss gp: -3.526
Loss gp: -3.527
Loss gp: -3.528
Loss gp: -3.529
Loss gp: -3.528
Loss gp: -3.529
Loss gp: -3.529
Loss gp: -3.530
Loss gp: -3.529
Loss gp: -3.530
Loss gp: -3.531
Loss gp: -3.531
Loss gp: -3.531
Loss gp: -3.531
Loss gp: -3.531
Loss gp: -3.532
Loss gp: -3.531
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Loss gp: -3.532
Converge

Loss gp: -3.416
Loss gp: -3.425
Loss gp: -3.416
Loss gp: -3.425
Loss gp: -3.434
Loss gp: -3.442
Loss gp: -3.434
Loss gp: -3.442
Loss gp: -3.450
Loss gp: -3.458
Loss gp: -3.450
Loss gp: -3.458
Loss gp: -3.465
Loss gp: -3.475
Loss gp: -3.465
Loss gp: -3.475
Loss gp: -3.486
Loss gp: -3.491
Loss gp: -3.486
Loss gp: -3.491
Loss gp: -3.499
Loss gp: -3.505
Loss gp: -3.499
Loss gp: -3.505
Loss gp: -3.509
Loss gp: -3.514
Loss gp: -3.509
Loss gp: -3.514
Loss gp: -3.517
Loss gp: -3.522
Loss gp: -3.517
Loss gp: -3.522
Loss gp: -3.524
Loss gp: -3.527
Loss gp: -3.524
Loss gp: -3.527
Loss gp: -3.529
Loss gp: -3.531
Loss gp: -3.529
Loss gp: -3.531
Loss gp: -3.532
Loss gp: -3.533
Loss gp: -3.532
Loss gp: -3.533
Loss gp: -3.534
Loss gp: -3.535
Loss gp: -3.534
Loss gp: -3.535
Loss gp: -3.536
Loss gp: -3.537
Loss gp: -3.536
Loss gp: -3.537
Loss gp: -3.537
Loss gp: -3.538
Loss gp: -3.537
Loss gp: -3.538
Loss gp: -3.538
Loss gp: -3.539
Loss gp: -3.538
Loss gp: -3.539
Loss gp: -3.539
Loss gp: -3.539
Loss gp:

Loss gp: 0.755
Loss gp: 0.742
Loss gp: 0.755
Loss gp: 0.742
Loss gp: 0.716
Loss gp: 0.663
Loss gp: 0.716
Loss gp: 0.663
Loss gp: 0.554
Loss gp: 0.319
Loss gp: 0.554
Loss gp: 0.319
Loss gp: -0.192
Loss gp: -1.276
Loss gp: -0.192
Loss gp: -1.276
Loss gp: -3.123
Loss gp: -3.413
Loss gp: -3.123
Loss gp: -3.413
Loss gp: -3.419
Loss gp: -3.424
Loss gp: -3.419
Loss gp: -3.424
Loss gp: -3.432
Loss gp: -3.438
Loss gp: -3.432
Loss gp: -3.438
Loss gp: -3.445
Loss gp: -3.451
Loss gp: -3.445
Loss gp: -3.451
Loss gp: -3.457
Loss gp: -3.463
Loss gp: -3.457
Loss gp: -3.463
Loss gp: -3.470
Loss gp: -3.476
Loss gp: -3.470
Loss gp: -3.476
Loss gp: -3.483
Loss gp: -3.489
Loss gp: -3.483
Loss gp: -3.489
Loss gp: -3.495
Loss gp: -3.501
Loss gp: -3.495
Loss gp: -3.501
Loss gp: -3.505
Loss gp: -3.505
Loss gp: -3.511
Loss gp: -3.516
Loss gp: -3.511
Loss gp: -3.516
Loss gp: -3.521
Loss gp: -3.521
Loss gp: -3.525
Loss gp: -3.525
Loss gp: -3.528
Loss gp: -3.528
Loss gp: -3.531
Loss gp: -3.531
Loss gp: -3.533
Loss

In [ ]:
#x0 = Tensor(np.array([0.1937, 0.1257]))
#x0 = Tensor(np.array([0.1885, 0.1038]))

x0 = Tensor(np.array([0.2000, 0.1002]))
print(vf(x0))
x0 = x0.reshape(1,2)
print(x0)
model.eval()
likelihood.eval()

pr = likelihood(model(x0))
print(pr.mean)

In [ ]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = Tensor(np.random.random_sample((50,2)))
    predictions = likelihood(model(test_x))
    mean = predictions.mean
    lower, upper = predictions.confidence_region()
    
plt.plot(mean, 'bo')
plt.plot(vfield_(test_x), 'r+')
plt.plot(upper, 'gx')
plt.plot(lower, 'mx')
plt.show()

In [ ]:
iter
print(agg_data.shape)

In [ ]:
res = 0
res = res + torch.ones(10,10)

In [ ]:
print(g_theta2_vec.shape)
print(iter)

In [ ]:
# v2 = g_theta2_vec.reshape(19,2)
# print(vec_x)

In [ ]:
# print(v2)
# np.savetxt('g_theta2_sameopti_keeping.txt', v2.detach().numpy())

In [ ]:

# np.savetxt('vec_x_sameopti_keeping.txt', vec_x.detach().numpy())

In [ ]:
# class design_opti_pll(nn.Module):
#     def __init__(self, x):
#         super(design_opti_pll, self).__init__()
#         #loc = np.random.random_sample((loc_size,2))
#         #self.g_theta2 = nn.Parameter(Tensor(sample))
#         self.x_design = nn.Parameter(Tensor(x))
#     def forward(self):
       
#         #g_theta2_new = self.g_theta2 #filter_sample(self.g_theta2, 0.009)
        
#         return self.x_design

In [ ]:
# def conduct_design_pll(x0,f_target, g_theta, agg_data, model, likelihood, training_design_iter, training_param_iter, lr_new):
#     design = design_opti_pll(x0)
#     loc_sample0 = loc_sample
#     x_d = design.forward()
#     def closure():
#         optimizer.zero_grad()
        
#         loss2,lower_bound, upper_bound = likelihood.get_pll(f_target,x_d, g_theta, agg_data, model, likelihood)
#         #loss2 = -1. * loss2
#         loss2.backward(retain_graph=True)
# #         print(x_d)
# #         print(lower_bound)
# #         print(upper_bound)
       
#         return loss2
        
        
        
#     optimizer = torch.optim.LBFGS(design.parameters(), lr=lr_new, history_size=100, max_iter=100, line_search_fn="strong_wolfe")
#     optimizer.step(closure)

#     x_d = design.forward()
#     loss2,lower_bound, upper_bound = likelihood.get_pll(f_target,x_d, g_theta, agg_data, model, likelihood)
#     #loss2 = -1. * loss2
#     print('Loss design: %.3f' % ( loss2))
#    # print(optimizer.state_dict())
#     print(x_d)
#     return x_d, lower_bound, upper_bound

In [ ]:
# iter_hp = 100
# iter_design = 40 
# iter_param = 50
# num_base_kernels = 5

# f_target = Tensor(vf.tgt_vec) 
# f_target = f_target.reshape(f_target.shape[0],1) 
# tol_vector = 0.001 * torch.ones(f_target.shape)


# loc_size = 200
# loc_sample = np.random.random_sample((loc_size,2))
# g_theta_ = (Tensor(loc_sample).clone())
# agg_data1 = vfield_(g_theta_)
# agg_data1 = agg_data1.flatten()


# x0 = Tensor(np.array([0.5,0.7])) 
# x0 = x0.reshape(1,2)
# x00 = x0 
# vec_x = Tensor(np.array([0.5,0.7])) 
# vec_x = vec_x.reshape(1,2)

# lr_new = 1.

# SUCCESS = False 
# FAILURE = False 
 
# tol = 0.009 
# print('START HYPERPARAMETERS optimization')
# model, likelihood = hyper_opti(g_theta_,agg_data1,iter_hp,num_base_kernels)

# print('END HYPERPARAMETERS optimization')

# x0_new,lower_bound, upper_bound = conduct_design_pll(x0,f_target, g_theta_, agg_data1, model, likelihood, iter_design, iter_param, lr_new)
# print(lower_bound)
# print(upper_bound)
# print(f_target-tol_vector)
# print(f_target+tol_vector)
# loc_sample = np.random.random_sample((loc_size,2))


# SUCCESS, FAILURE = stopping_criteria(tol_vector, f_target, lower_bound, upper_bound)


# print(x0_new)